In [ ]:
# Import library
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load image
def load_image(path):
  img = cv2.imread(path)

  if img is None:
    raise FileNotFoundError(f"Image not found: {path}")

  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

# Grayscale
def to_grayscale(img):
  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  return gray

# Gaussian Blur
def apply_gaussian_blur(gray, ksize=(5, 5), sigma=1):
  blur = cv2.GaussianBlur(gray, ksize, sigma)
  return blur

# Canny Edge Detection
def apply_canny(gray, low=100, high=200):
  edges = cv2.Canny(gray, low, high)
  return edges

# Hough Circle Detection
def detect_circles(gray):
  circles = cv2.HoughCircles(
    gray,
    cv2.HOUGH_GRADIENT,
    dp=1.2,
    minDist=40,
    param1=100,
    param2=30,
    minRadius=20,
    maxRadius=120
  )
  return circles

# Main program
path = "tests/test1.png"
image = load_image(path)

gray = to_grayscale(image)
blur = apply_gaussian_blur(gray)
edges = apply_canny(blur)
circles = detect_circles(blur)

output = image.copy()
coin_count = 0

if circles is not None:
  circles = np.round(circles[0, :]).astype("int")
  coin_count = len(circles)

  for (x, y, r) in circles:
    cv2.circle(output, (x, y), r, (0, 255, 0), 3)
    cv2.circle(output, (x, y), 2, (255, 0, 0), 3)

print("Jumlah koin terdeteksi:", coin_count)

# Tampilkan gambar
plt.figure(figsize=(20, 5))

plt.subplot(1, 4, 1)
plt.title("Original")
plt.imshow(image)
plt.axis("off")

plt.subplot(1, 4, 2)
plt.title("Grayscale")
plt.imshow(gray, cmap="gray")
plt.axis("off")

plt.subplot(1, 4, 3)
plt.title("Gaussian Blur")
plt.imshow(blur, cmap="gray")
plt.axis("off")

plt.subplot(1, 4, 4)
plt.title(f"Detected Coins: {coin_count}")
plt.imshow(output)
plt.axis("off")

plt.show()


Jumlah koin: 4237
